# Task 2: Text Chunking, Embedding, and Vector Store Indexing

In [4]:
# %pip install faiss-cpu

In [5]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import os
import pickle
import sys
sys.path.append(os.path.abspath("../src"))
import warnings
warnings.filterwarnings('ignore')

In [6]:
# %%
os.chdir("..")  # Go up a directory
#print(os.getcwd())

In [7]:
from Embeding_Index import *

In [8]:
# Access Function
def run_pipeline(input_path="Data/filtered_complaints.csv", chunk_size=300, chunk_overlap=50):
    df = load_cleaned_data(input_path)
    if df is None:
        return

    print(" Chunking text...")
    texts, metadata = chunk_texts(df, chunk_size, chunk_overlap)

    print(" Generating embeddings...")
    embeddings = generate_embeddings(texts)

    print("Indexing to FAISS...")
    index_with_faiss(embeddings, metadata)

if __name__ == "__main__":
    run_pipeline()

 Chunking text...
 Generating embeddings...


Batches: 100%|██████████| 45/45 [00:25<00:00,  1.79it/s]

Indexing to FAISS...
Vector store saved in vector_store/faiss_index
